1. Mover todos os arquivos HTML da pasta HTTrack_utfpr.edu.br/www.utfpr.edu.br para a pasta Files/HTML/www.utfpr.edu.br

2. Gerar arquivo CSV com todos os links que direcionam para documentos publicados no SEI.

3. Para cada link do SEI presenta no CSV, salvar um arquivo HTML na pasta Files/HTML/SEI

4. Mover todos os arquivos PDF da pasta HTTrack_utfpr.edu.br/www.utfpr.edu.br para a pasta Files/HTML/www.utfpr.edu.br

In [ ]:
import os
import shutil

# Diretório base do projeto (a pasta TCC)
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Caminhos das pastas relativos ao projeto
source_dir = os.path.join(base_dir, "Webscraping", "Files", "HTTrack_utfpr.edu.br", "www.utfpr.edu.br")
destination_dir = os.path.join(base_dir, "Webscraping", "Files", "HTML", "www.utfpr.edu.br")

# Função para copiar arquivos mantendo a estrutura de pastas
def copy_html_files(source, destination):
    for root, _, files in os.walk(source):
        for file in files:
            if file.endswith(".html"):
                # Caminho completo do arquivo de origem
                source_file = os.path.join(root, file)
                
                # Caminho relativo para manter a estrutura
                relative_path = os.path.relpath(root, source)
                
                # Caminho completo do destino
                destination_folder = os.path.join(destination, relative_path)
                destination_file = os.path.join(destination_folder, file)
                
                # Cria as pastas no destino, se necessário
                os.makedirs(destination_folder, exist_ok=True)
                
                # Copia o arquivo
                shutil.copy2(source_file, destination_file)
                print(f"Copiado: {source_file} -> {destination_file}")

# Executa a função
copy_html_files(source_dir, destination_dir)

In [ ]:
import os
import csv
from bs4 import BeautifulSoup

# Diretório base do projeto (a pasta TCC)
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Caminho do diretório onde os arquivos HTML foram copiados
destination_dir = os.path.join(base_dir, "Webscraping", "Files", "HTML", "www.utfpr.edu.br")

# Caminho para salvar o arquivo links_SEI.csv
output_csv = os.path.join(base_dir, "Webscraping", "links_SEI.csv")

# Função para extrair links que começam com o prefixo desejado
def extract_links_with_paths(source, prefix):
    links_with_paths = []
    for root, _, files in os.walk(source):
        for file in files:
            if file.endswith(".html"):
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, source)  # Caminho relativo ao diretório base
                with open(file_path, "r", encoding="utf-8") as f:
                    soup = BeautifulSoup(f, "html.parser")
                    for a_tag in soup.find_all("a", href=True):
                        href = a_tag["href"]
                        if href.startswith(prefix):
                            links_with_paths.append((href, relative_path))
    return links_with_paths

# Função para salvar os links e caminhos no arquivo CSV
def save_links_with_paths_to_csv(links_with_paths, output_file):
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile, delimiter=";")  # Usa ponto e vírgula como separador
        writer.writerow(["Link", "Caminho"])  # Cabeçalho do CSV
        for link, path in links_with_paths:
            writer.writerow([link, path])
    print(f"Arquivo CSV gerado: {output_file}")

# Prefixo dos links que queremos extrair
link_prefix = "https://sei.utfpr.edu.br/sei/publicacoes/"

# Extrai os links e os caminhos relativos dos arquivos HTML
links_with_paths = extract_links_with_paths(destination_dir, link_prefix)

# Salva os links e caminhos no arquivo CSV
save_links_with_paths_to_csv(links_with_paths, output_csv)

In [ ]:
import os
import requests
from tqdm import tqdm
import re
import csv

# Diretório base do projeto (a pasta TCC)
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Caminho para salvar os arquivos HTML baixados
sei_dir = os.path.join(base_dir, "Webscraping", "Files", "HTML", "SEI")
os.makedirs(sei_dir, exist_ok=True)  # Cria a pasta SEI, se não existir

# Caminho do arquivo CSV com os links e caminhos
input_csv = os.path.join(base_dir, "Webscraping", "links_SEI.csv")

# Função para gerar um nome de arquivo seguro
def sanitize_filename(link):
    # Substitui caracteres inválidos por "_"
    return re.sub(r'[<>:"/\\|?*]', '_', link.split("/")[-1] or "index.html")

# Função para baixar e salvar os arquivos HTML obedecendo à estrutura de pastas
def download_html_with_structure(csv_file, destination):
    with open(csv_file, "r", encoding="utf-8") as csvfile:
        reader = csv.reader(csvfile, delimiter=";")
        next(reader)  # Pula o cabeçalho
        for row in tqdm(reader, desc="Baixando arquivos HTML"):
            if len(row) < 2:
                continue  # Ignora linhas incompletas
            link, relative_path = row
            try:
                response = requests.get(link, verify=False, stream=True)  # Ignora a verificação SSL e usa stream
                response.raise_for_status()  # Verifica se houve erro na requisição
                
                # Gera um nome de arquivo seguro
                file_name = sanitize_filename(link)
                if not file_name.endswith(".html"):
                    file_name += ".html"  # Garante que o arquivo tenha a extensão .html
                
                # Cria a estrutura de pastas com base no caminho relativo
                folder_path = os.path.join(destination, relative_path)
                os.makedirs(folder_path, exist_ok=True)
                
                # Caminho completo do arquivo
                file_path = os.path.join(folder_path, file_name)
                
                # Salva o conteúdo HTML no arquivo
                with open(file_path, "wb") as f:  # Salva em modo binário para preservar o conteúdo original
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                # print(f"Baixado: {link} -> {file_path}")
            except requests.RequestException as e:
                print(f"Erro ao baixar {link}: {e}")

# Faz o download dos arquivos HTML obedecendo à estrutura de pastas
download_html_with_structure(input_csv, sei_dir)

In [ ]:
import os
import shutil

# Diretório base do projeto (a pasta TCC)
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Caminhos das pastas relativos ao projeto
source_dir = os.path.join(base_dir, "Webscraping", "Files", "HTTrack_utfpr.edu.br", "www.utfpr.edu.br")
destination_dir = os.path.join(base_dir, "Webscraping", "Files", "PDF", "www.utfpr.edu.br")

# Função para copiar arquivos PDF mantendo a estrutura de pastas
def copy_pdf_files(source, destination):
    for root, _, files in os.walk(source):
        for file in files:
            if file.endswith(".pdf"):
                # Caminho completo do arquivo de origem
                source_file = os.path.join(root, file)
                
                # Caminho relativo para manter a estrutura
                relative_path = os.path.relpath(root, source)
                
                # Caminho completo do destino
                destination_folder = os.path.join(destination, relative_path)
                destination_file = os.path.join(destination_folder, file)
                
                # Cria as pastas no destino, se necessário
                os.makedirs(destination_folder, exist_ok=True)
                
                # Copia o arquivo
                shutil.copy2(source_file, destination_file)
                print(f"Copiado: {source_file} -> {destination_file}")

# Executa a função
copy_pdf_files(source_dir, destination_dir)